# Installation

In [ ]:
pip install path4gmns

# Prepare Data Set

You can either download the sample data sets with the built-in function or prepare your own data set. node.csv and link.csv complying GMNS is the minimum requirement for you to get started.

In [ ]:
import path4gmns as pg

In [ ]:
pg.download_sample_data_sets()

Navigate to the target data set directory. Here we will use the data set where this Jupyter notebook is located.

In [ ]:
pwd

# The Basic Functionalties
## Get the Shortest Path between Two Nodes
Find the (static) shortest path (based on distance) and output it in the format of a sequence of node/link IDs.

In [ ]:
network = pg.read_network(load_demand=False)

print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2))
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, seq_type='link'))

## Find Shortest Paths for All Individual Agents

Agents are disaggreated demand using the aggregated travel demand between each OD pair, which is specified in demand.csv. Individual agents will be automatically set up via find_path_for_agents() on its first call.
### If you have demand.csv

In [ ]:
network = pg.read_network()
network.find_path_for_agents()

agent_id = 300
print('\norigin node id of agent is '
      f'{network.get_agent_orig_node_id(agent_id)}')
print('destination node id of agent is '
      f'{network.get_agent_dest_node_id(agent_id)}')
print('shortest path (node id) of agent, '
      f'{network.get_agent_node_path(agent_id)}')
print('shortest path (link id) of agent, '
      f'{network.get_agent_link_path(agent_id)}')

agent_id = 1000
print('\norigin node id of agent is '
      f'{network.get_agent_orig_node_id(agent_id)}')
print('destination node id of agent is '
      f'{network.get_agent_dest_node_id(agent_id)}')
print('shortest path (node id) of agent, '
      f'{network.get_agent_node_path(agent_id)}')
print('shortest path (link id) of agent, '
      f'{network.get_agent_link_path(agent_id)}')

# output unique agent paths to a csv file
# if you do not want to include geometry info in the output file,
# use pg.output_agent_paths(network, False)
pg.output_agent_paths(network)

### If you do not have demand.csv
Path4GMNS can synthesize zones and OD demand matrix for you.
#### Synthesize zones and OD demand matrix

In [ ]:
network = pg.read_network(load_demand=False)

# by default, grid_dimension is 8, total_demand is 10,000,
# time_budget is 120 min, mode is 'auto'
pg.network_to_zones(network)
pg.output_zones(network)
pg.output_synthesized_demand(network)

#### Load the synthesized zones and OD demand matrix

In [ ]:
pg.read_zones(network)
pg.load_demand(network, filename='syn_demand.csv')

#### Find Shortest Paths for All Individual Agents

In [ ]:
network.find_path_for_agents()

# output unique agent paths to a csv file
# if you do not want to include geometry info in the output file,
# use pg.output_agent_paths(network, False)
pg.output_agent_paths(network)

## Perform Path-Based UE Traffic Assignment

Similar to Finding Shortest Paths for All Individual Agents, OD demand matrix is also need to perform this functionality.

Assume you have successfully load the demand via either pg.read_network() in cell 5 or pg.load_demand(network, filename='syn_demand.csv') in cell 8. Then you can conduct traffic assignment via the following lines.

In [ ]:
column_gen_num = 20
column_update_num = 10

# path-based UE only
pg.perform_column_generation(column_gen_num, column_update_num, network)

# if you do not want to include geometry info in the output file,
# use pg.output_columns(network, False)
pg.output_columns(network)
pg.output_link_performance(network)

# Move Foward to Multimodal Evaluation

In order to perform multimodal evaluation, the corresponding modes (i.e., agent types) must be presented in settings.yml. It will be parsed by pyyaml (5.1 or higher) to the Python engine at run-time. A sample file looks like blow.

```yaml
agents:
  - type: a
    name: auto
    vot: 10
    flow_type: 0
    pce: 1
    free_speed: 60
    use_link_ffs: true
  - type: w
    name: walk
    vot: 10
    flow_type: 0
    pce: 1
    free_speed: 10
    use_link_ffs: false

demand_periods:
  - period: AM
    time_period: 0700_0800

demand_files:
  - file_name: demand.csv
    format_type: column
    period: AM
    agent_type: a
```

## Get the Shortest Path between Two Nodes under a Specific Mode
In cell 4, we use this functionlity for the default mode, which is 'a' or equivalently 'auto'. Now with 'walk' defined in settings.yml, we are able to find the shortest path under mode 'w' or 'walk'.

In [ ]:
network = pg.read_network(load_demand=False)

print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, mode='w'))
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, mode='w', seq_type='link'))

For this test data set, cells 4 and 12 have the same results as each link is open to all modes, i.e., their "allowed_uses" are "all".  

In [ ]:
network = pg.read_network()
network.find_path_for_agents()

# or equivalently network.find_path_for_agents('walk')
network.find_path_for_agents('w')

# retrieving the origin, the destination, and the shortest path of a given agent
# is exactly the same as before as well as outputting all unique agent paths
pg.output_agent_paths(network)